# Compiling Expressions

A key benefit of having a programmatic representations of shapes is the ability use static analysis to optimize how the program is executed. This notebooks shows how we use expression compilation to speed up execution of batches of GeoLIPI expressions by an order of magnitude.

In [1]:
import os
import torch as th
import inspect
import random
import time
import numpy as np
import _pickle as cPickle

import geolipi.symbolic as gls
from geolipi.torch_compute import Sketcher, expr_to_sdf, recursive_evaluate
# from geolipi.torch_compute import create_compiled_expr, create_evaluation_batches, batch_evaluate
from geolipi.torch_compute.compile_expression import create_compiled_expr
from geolipi.torch_compute.batch_evaluate_sdf import create_evaluation_batches, batch_evaluate

In [3]:
# Sketcher keeps track of some tensor device and resolution
resolution = 64
sketcher = Sketcher(device="cuda", resolution=resolution, n_dims=3)

In [4]:
# Load a bunch of example programs
# Programs are saved in string format, and we use python's eval to convert them to a gls expressions
# NOTE - it can be dangerous to use eval on strings you don't trust.
all_functions = []

str_to_cmd_mapper = gls.get_cmd_mapper()
str_to_cmd_mapper['tensor'] = th.tensor

file_name = "../assets/example_random_programs.pkl"
with open(file_name, 'rb') as f:
    expressions = cPickle.load(f)
expressions = [eval(p, str_to_cmd_mapper) for p in expressions]
random.shuffle(expressions)
print(f"Loaded {len(expressions)} expressions")


Loaded 5120 expressions


In [5]:
# Naive recursive execution:
# Supports all the gls expressions, but slow.
start_time = time.time()
for ind, expression in enumerate(expressions):
    if ind % 1000 == 0:
        print(f"Took {time.time() - start_time} seconds to run {ind} programs")
    cuda_expression = expression.cuda()
    sdf = recursive_evaluate(cuda_expression, sketcher)
end_time = time.time()
print("Naive recursive execution time: ", end_time - start_time)

Took 7.176399230957031e-05 seconds to run 0 programs
Took 8.823237895965576 seconds to run 1000 programs
Took 17.36352300643921 seconds to run 2000 programs
Took 25.824536561965942 seconds to run 3000 programs
Took 34.41025185585022 seconds to run 4000 programs
Took 43.108124017715454 seconds to run 5000 programs
Naive recursive execution time:  44.037843227386475


In [6]:
# expr_to_sdf uses stack based parsing to avoid recursion.
# but DOES NOT support higher level primitives, and some of the transforms (it only keeps track of the affine transform matrix not coords)
start_time = time.time()
for ind, expression in enumerate(expressions):
    if ind % 1000 == 0:
        print(f"Took {time.time() - start_time} seconds to run {ind} programs")
    cuda_expression = expression.cuda()
    sdf = expr_to_sdf(cuda_expression, sketcher)
end_time = time.time()
print("Simple iterative execution time: ", end_time - start_time)

Took 4.5299530029296875e-05 seconds to run 0 programs
Took 8.583478212356567 seconds to run 1000 programs
Took 16.980957508087158 seconds to run 2000 programs
Took 25.22557306289673 seconds to run 3000 programs
Took 33.60206961631775 seconds to run 4000 programs
Took 42.07800793647766 seconds to run 5000 programs
Simple iterative execution time:  42.96791124343872


In [5]:
# Second way is to use batching. Even including batching time, this is often slightly faster.
# first for each expression, we need to convert it into a compiled form
# Lets do it in batches of 256
batch_size = 64
n_batches = int(np.ceil(len(expressions) / batch_size))
start_time = time.time()
for cur_batch in range(n_batches):
    cur_expressions = expressions[cur_batch * batch_size : (cur_batch + 1) * batch_size]
    eval_stack = []
    for expr in cur_expressions:
        cuda_expr = expr.cuda()
        expr, transforms, inversions, params = create_compiled_expr(cuda_expr, sketcher, convert_to_cpu=False)
        eval_stack.append([expr, transforms, inversions, params])

    eval_batches = create_evaluation_batches(eval_stack, convert_to_cuda=False)
    all_sdfs = batch_evaluate(eval_batches, sketcher)
    
    if cur_batch % 10 == 0:
        print(f"Took {time.time() - start_time} seconds to run {cur_batch} batches of size {batch_size}")
end_time = time.time()
print("Batched execution time: ", end_time - start_time)

Took 0.6157186031341553 seconds to run 0 batches of size 64
Took 5.344455718994141 seconds to run 10 batches of size 64
Took 10.854475736618042 seconds to run 20 batches of size 64
Took 16.030340433120728 seconds to run 30 batches of size 64
Took 20.73273468017578 seconds to run 40 batches of size 64
Took 25.84080982208252 seconds to run 50 batches of size 64
Took 31.261850357055664 seconds to run 60 batches of size 64
Took 35.96721577644348 seconds to run 70 batches of size 64
Batched execution time:  40.16727304458618


In [6]:
# more importantly, we can load the compiled versions into a memory, and simply load and execute them when required.
# This can speed up the execution time by a lot, especially when there are many programs to execute.
# This version of compile + execute is integrated in the code base for [CoReF]().
# Second way is to use batching. Even including batching time, this is often slightly faster.
# first for each expression, we need to convert it into a compiled form
# Lets do it in batches of 256
batch_size = 64
n_batches = int(np.ceil(len(expressions) / batch_size))
start_time = time.time()
eval_stack = []
for ind, expr in enumerate(expressions):
    cuda_expr = expr.cuda()
    expr, transforms, inversions, params = create_compiled_expr(cuda_expr, sketcher, convert_to_cpu=True)
    eval_stack.append([expr, transforms, inversions, params])

    if ind % 1000 == 0:
        print(f"Took {time.time() - start_time} seconds to compile {ind} programs")
end_time = time.time()
print("Compilation time: ", end_time - start_time)

start_time = time.time()
for cur_batch in range(n_batches):
    cur_eval = eval_stack[cur_batch * batch_size : (cur_batch + 1) * batch_size]
    eval_batches = create_evaluation_batches(cur_eval, convert_to_cuda=True)
    all_sdfs = batch_evaluate(eval_batches, sketcher)
    
    if cur_batch % 10 == 0:
        print(f"Took {time.time() - start_time} seconds to run {cur_batch} batches of size {batch_size}")
end_time = time.time()
print("Compiled Execution time: ", end_time - start_time)

# This yields a significant speedup ~ 500 %. 

Took 0.0070345401763916016 seconds to compile 0 programs
Took 6.822115898132324 seconds to compile 1000 programs
Took 13.309079885482788 seconds to compile 2000 programs
Took 19.822893619537354 seconds to compile 3000 programs
Took 26.366687297821045 seconds to compile 4000 programs
Took 32.809659004211426 seconds to compile 5000 programs
Compilation time:  33.46465611457825
Took 0.06953549385070801 seconds to run 0 batches of size 64
Took 0.8522241115570068 seconds to run 10 batches of size 64
Took 1.6405560970306396 seconds to run 20 batches of size 64
Took 2.439478635787964 seconds to run 30 batches of size 64
Took 3.1721460819244385 seconds to run 40 batches of size 64
Took 4.003402471542358 seconds to run 50 batches of size 64
Took 4.820006370544434 seconds to run 60 batches of size 64
Took 5.565735816955566 seconds to run 70 batches of size 64
Compiled Execution time:  6.244786500930786


In [5]:
# Extra (NOT RECOMMENDED) - It is also possible to convert the programs into a CNF/DNF form, and then execute them.
# This can make all the expression have a single union, and multiple intersection operators - which can sometimes be much faster (if the intersections are batched)
# This DNF form of expressions is used in [CSGStump]().

batch_size = 64
n_batches = int(np.ceil(len(expressions) / batch_size))
eval_stack = []

start_time = time.time()
for ind, expr in enumerate(expressions):
    cuda_expr = expr.cuda()
    expr, transforms, inversions, params = create_compiled_expr(cuda_expr, sketcher, convert_to_cpu=True, resolve_to_dnf=True)
    eval_stack.append([expr, transforms, inversions, params])

    if ind % 1000 == 0:
        print(f"Took {time.time() - start_time} seconds to compile {ind} programs")
end_time = time.time()
print("Compilation time: ", end_time - start_time)

start_time = time.time()
for cur_batch in range(n_batches):
    cur_eval = eval_stack[cur_batch * batch_size : (cur_batch + 1) * batch_size]
    eval_batches = create_evaluation_batches(cur_eval, convert_to_cuda=True)
    all_sdfs = batch_evaluate(eval_batches, sketcher)
    
    if cur_batch % 10 == 0:
        print(f"Took {time.time() - start_time} seconds to run {cur_batch} batches of size {batch_size}")
end_time = time.time()
print("Compiled Execution time for dnf programs: ", end_time - start_time)


Took 0.06584429740905762 seconds to compile 0 programs
Took 22.122634410858154 seconds to compile 1000 programs
Took 40.67112684249878 seconds to compile 2000 programs
Took 62.87722945213318 seconds to compile 3000 programs
Took 85.7822482585907 seconds to compile 4000 programs
Took 108.93477153778076 seconds to compile 5000 programs
Compilation time:  110.41778016090393
Took 0.2236177921295166 seconds to run 0 batches of size 64
Took 3.050459861755371 seconds to run 10 batches of size 64
Took 6.006987810134888 seconds to run 20 batches of size 64
Took 8.54468059539795 seconds to run 30 batches of size 64
Took 11.858993768692017 seconds to run 40 batches of size 64
Took 14.960047006607056 seconds to run 50 batches of size 64
Took 17.59245204925537 seconds to run 60 batches of size 64
Took 20.26682448387146 seconds to run 70 batches of size 64
Compiled Execution time for dnf programs:  22.381612062454224
